Below are parameters of the experiments reported in [KLNO24](https://eprint.iacr.org/2024/1972).

In [14]:
from rok_estimator.rok_estimator import *

kjl = 8


g_first_branch = None
g_second_branch = None

g_second_branch_ext_2_norm = None
g_second_branch_ext_inf_norm = None

class Relation_RP(Relation):
    def execute(self, op, **kwargs):
        match op:
            case "srp":
                return self.pi_srp(**kwargs)
            case "to-first":
                return self.pi_to_first(**kwargs)
            case "join":
                return self.pi_join(**kwargs)
            case _:
                return super().execute(op, **kwargs)

            
    def pi_srp(self):
        global g_first_branch 
        rel_params = {
            "n_rel": self.n_rel + 1,
            "rep": 1,
        }

        g_first_branch = replace(deepcopy(self), **rel_params)
        comm = (self.n_commit + 1) * self.rep * self.ring.size_Rq()  # for the new comm
        snd_err = 0 # wrong
        print(self.wdim)
        global g_second_branch_ext_2_norm
        global g_second_branch_ext_inf_norm

        def log_beta_ext_2_func(x):
            global g_second_branch_ext_2_norm
            return g_second_branch_ext_2_norm
        def log_beta_ext_inf_func(x):
            global g_second_branch_ext_inf_norm
            return g_second_branch_ext_inf_norm
            
        rel_params = {
            "op_name": "srp",
            "n_compress": self.n_commit + 1,
            "n_rel": 1,
            "wdim": ZZ(self.wdim / kjl),
            "comm": comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,           
            "log_beta_ext_2_func" : log_beta_ext_2_func, # we extract norm from the second branch (relax it)
            "log_beta_ext_inf_func" : log_beta_ext_inf_func # let it be , wrong
        }      # We switch to the second branch immediately to estimate binding and to serve as an extraction checkpoint. Let's assume for now that the norm does not degrade
        return replace(self, **rel_params)

    def pi_to_first(self):
        global g_second_branch 
        g_second_branch = deepcopy(self)
        
        rel_params = {
            "op_name": "switch 1",
            "n_rel": 1,
            "log_beta_ext_2_func" : lambda x : x, # perfect extraction
            "log_beta_ext_inf_func" : lambda x : x # perfect extraction
        }      
        global g_first_branch
        return replace(g_first_branch, **rel_params)
        
    def pi_join(self):
        def log_beta_ext_2_func(x):
            global g_second_branch_ext_2_norm
            g_second_branch_ext_2_norm = x
            return x
        def log_beta_ext_inf_func(x):
            global g_second_branch_ext_inf_norm
            g_second_branch_ext_inf_norm = x
            return x

        global g_second_branch
        rel_params = {
            "op_name": "join",
            "n_rel": self.n_rel + g_second_branch.n_rel,
            "log_beta_wit_2": max(self.log_beta_wit_2, g_second_branch.log_beta_wit_2),
            "log_beta_wit_inf": max(self.log_beta_wit_inf, g_second_branch.log_beta_wit_inf),
            "log_beta_ext_2_func": log_beta_ext_2_func, # perfect extraction but we keep track of what was there
            "log_beta_ext_inf_func" : log_beta_ext_inf_func # perfect extraction
        }      
        return replace(g_first_branch, **rel_params)
        
        
    

In [15]:
ring_params = {
    "f": 60,
    "log_beta_sis_2": 44,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**16,
    "rep": 1,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "ell": 3,
    "d": 2,
}

loop = [
    ("norm", {}), 
    ("batch", {}), 
    ("bdecomp", {"ell": ops_params["ell"]}), 
    ("srp", {}), # switches to the second branch to check hardness
    ("to-first", {}),
    ("split", {"d": ops_params["d"]}), 
    ("fold", {}),
    ("join", {})
]

ops = loop * 10 + [("finish", {})]

sim = Simulation(ring_params, rel_params, Relation_RP)
sim.ring.C = SubtractiveSet(cardinality = 2^100, gamma_2 = 10, theta_2 = 100, gamma_inf = 100, theta_inf = 17) # guess


sim.execute(ops)
sim.extract()
sim.show()

65536
65536
65536
65536
65536
65536
65536
65536
65536
65536
Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |    65536 |   1 |    ( 13 | 13/  0 )        |     (  0 | 13/  0 )         | 128.0 KB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |    65536 |  14 |    ( 13 | 28/  0 )        |     (  0 | 28/  0 )         | 1.750 MB |      (125.5 KB | 125.5 KB)      |         (2^-124 | 2^-124)         
 batch     |    65536 |  14 |    ( 13 | 28/  0 )        |     (  0 | 28/  0 )         | 1.750 MB |      (125.5 KB | 125.5 KB)      |         (2^-122 | 2^-122)         
 bdecomp   |    65536 |  42 |    ( 13 | 25/  0 )        |     (  0 | 25/  0 )         | 5.250 MB |      (262.5 KB | 388.0 KB)      |         (2^-oo  | 2^-122)         
 srp       |     8192 |  42 |    ( 11 | 33/  0 )        |     (  0 | 30/  0 )      